In [ ]:
import pandas as pd
import requests
import time
from datetime import datetime, timedelta
import sys

excel_filename = 'ML DATASET.xlsx'
target_sheets = ['BORNEO(MY)_DATA']
OUTPUT_FILE = 'final_borneo_malaysia_data.csv' 
SAMPLE_SIZE = 10000

print(f"Loading sheets: {target_sheets}...")
df_list = []

try:
    for sheet in target_sheets:
        print(f" Reading {sheet}...")
        df_sheet = pd.read_excel(excel_filename, sheet_name=sheet)
        df_list.append(df_sheet)

    df = pd.concat(df_list, ignore_index=True)
    print(f"✅ Combined Total Rows: {len(df)}")

except Exception as e:
    print(f"❌ Error loading Excel: {e}")
    sys.exit()

# --- sample size ---
if len(df) > SAMPLE_SIZE:
    print(f"⚠️ TEST MODE: Sampling {SAMPLE_SIZE} rows only...")
    df_final = df.sample(n=SAMPLE_SIZE, random_state=42)
else:
    df_final = df.copy()

print(f"Processing {len(df_final)} rows...")

# --- data ---
def get_raw_7days_weather(lat, lon, fire_date_str):
    try:
        if isinstance(fire_date_str, pd.Timestamp):
            date_obj = fire_date_str
        else:
            date_obj = datetime.strptime(str(fire_date_str).split(' ')[0], '%Y-%m-%d')

        date_keys = []
        for i in range(7, -1, -1): 
            d = date_obj - timedelta(days=i)
            date_keys.append(d.strftime('%Y%m%d'))

        start_str = date_keys[0]  
        end_str = date_keys[-1]   
        
        base_url = "https://power.larc.nasa.gov/api/temporal/daily/point"
        params = {
            'parameters': 'T2M,PRECTOTCORR,RH2M,WS2M,GWETTOP,GWETROOT', 
            'community': 'AG',
            'longitude': lon,
            'latitude': lat,
            'start': start_str,
            'end': end_str,
            'format': 'JSON'
        }

        response = requests.get(base_url, params=params, timeout=25)
        
        if response.status_code == 200:
            data = response.json()
            p = data['properties']['parameter']
            row_weather = {}

            for i, date_key in enumerate(date_keys):
                days_ago = 7 - i 
                
                val_t = p['T2M'].get(date_key, -999)
                row_weather[f'Temp_Day_{days_ago}'] = val_t if val_t > -100 else None
                
                val_r = p['PRECTOTCORR'].get(date_key, -999)
                row_weather[f'Rain_Day_{days_ago}'] = val_r if val_r >= 0 else None
                
                val_h = p['RH2M'].get(date_key, -999)
                row_weather[f'Humid_Day_{days_ago}'] = val_h if val_h >= 0 else None

                val_w = p['WS2M'].get(date_key, -999)
                row_weather[f'Wind_Day_{days_ago}'] = val_w if val_w >= 0 else None

                val_st = p['GWETTOP'].get(date_key, -999)
                row_weather[f'SoilSurf_Day_{days_ago}'] = val_st if val_st >= 0 else None

                val_sr = p['GWETROOT'].get(date_key, -999)
                row_weather[f'SoilRoot_Day_{days_ago}'] = val_sr if val_sr >= 0 else None
                
            return row_weather
        else:
            return None
    except:
        return None

# --- repeat ---
print("\nStart Fetching Data...")
weather_results = [] 

for index, row in df_final.iterrows():
    if len(weather_results) % 5 == 0:
        print(f"  Processed {len(weather_results)} / {len(df_final)} rows...")

    w_data = get_raw_7days_weather(row['latitude'], row['longitude'], row['acq_date'])
    
    if w_data:
        weather_results.append(w_data)
    else:
        weather_results.append({})
    
    time.sleep(0.3)

# --- excel ---
print("Merging and Grouping Columns...")
weather_df = pd.DataFrame(weather_results)

df_final = df_final.reset_index(drop=True)
weather_df = weather_df.reset_index(drop=True)
df_final = pd.concat([df_final, weather_df], axis=1)


feature_groups = ['Temp', 'Rain', 'Humid', 'Wind', 'SoilSurf', 'SoilRoot']
desired_weather_order = []

for feature in feature_groups:
    for day in range(7, -1, -1):
        col_name = f'{feature}_Day_{day}'
        desired_weather_order.append(col_name)

original_cols = [col for col in df_final.columns if col not in desired_weather_order]

existing_weather_cols = [c for c in desired_weather_order if c in df_final.columns]

final_order = original_cols + existing_weather_cols
df_final = df_final[final_order]

if 'Temp_Day_0' in df_final.columns:
    missing_count = df_final['Temp_Day_0'].isna().sum()
    print(f"Note: {missing_count} rows have missing weather data (Kept as empty cells).")

df_final.to_csv(OUTPUT_FILE, index=False, na_rep='')

print("\n" + "="*40)
print(f"DONE! Saved to: {OUTPUT_FILE}")
print("Columns are grouped by type: All Temps -> All Rains -> All Winds...")
print("="*40)

Loading sheets: ['BORNEO(MY)_DATA']...
 Reading BORNEO(MY)_DATA...
✅ Combined Total Rows: 8545
Processing 8545 rows...

Start Fetching Data...
  Processed 0 / 8545 rows...
  Processed 5 / 8545 rows...
  Processed 10 / 8545 rows...
  Processed 15 / 8545 rows...
  Processed 20 / 8545 rows...
  Processed 25 / 8545 rows...
  Processed 30 / 8545 rows...
  Processed 35 / 8545 rows...
  Processed 40 / 8545 rows...
